In [5]:
# Импортирую библеотеки
import sqlite3 as sql
import pandas as pd

# Настройка отображения DataFrame
pd.set_option('display.max_colwidth', None)


In [14]:
con = sql.connect(r"C:\Users\Димаш\Desktop\python\SQL Murder Mystery\sql-murder-mystery.db")
query1 = """
SELECT *
FROM crime_scene_report
WHERE city = 'SQL City' AND type = 'murder' AND date = 20180115
"""
pd.read_sql_query(query1, con)


,date,type,description,city
0,20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


In [ ]:
#Показания первого свидетеля
witness1 = """
select 
    interview.transcript,
    person.name,
    person.id
from interview
join person on interview.person_id=person.id
where person.address_street_name = 'Northwestern Dr'
order by person.address_number desc
limit 1
"""
pd.read_sql_query(witness1, con)


,transcript,name,id
0,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W"".",Morty Schapiro,14887


In [ ]:
#показания второго свидетеля 
witness2 = """
select 
    interview.transcript,
    person.name,
    person.id
from interview
join person on interview.person_id=person.id
where 
person.name like '%Annabel%' and person.address_street_name = 'Franklin Ave'
limit 1
"""
pd.read_sql_query(witness2, con)


,transcript,name,id
0,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.",Annabel Miller,16371


Clues:
Gold memeber in the gym
License plate on car was H42W
Membership number on bag start with 48Z
Had a "Get Fit Now" Gym bag


In [38]:
suspect= """
with gym as (
    select *
    from get_fit_now_member
    inner join get_fit_now_check_in
    on get_fit_now_check_in.membership_id = get_fit_now_member.id
    where 
        get_fit_now_member.membership_status = 'gold' 
        and 
        get_fit_now_check_in.check_in_date = 20180109
        and 
        get_fit_now_check_in.membership_id like '%48Z%'
        
)
select
person.name,
    person.address_number,
    person.address_street_name,
    person.ssn,
    drivers_license.car_make,
    drivers_license.car_model,
    drivers_license.age,
    drivers_license.height,
    drivers_license.eye_color,
    drivers_license.hair_color,
    drivers_license.gender,
    drivers_license.plate_number
from person
join gym on person.id = gym.person_id
join drivers_license on person.license_id = drivers_license.id
where drivers_license.plate_number like '%H42W%'
"""
pd.read_sql_query(suspect, con)

,name,address_number,address_street_name,ssn,car_make,car_model,age,height,eye_color,hair_color,gender,plate_number
0,Jeremy Bowers,530,"Washington Pl, Apt 3A",871539279,Chevrolet,Spark LS,30,70,brown,brown,male,0H42W2


In [44]:
jeremy_testimony = """
select transcript
from interview
join person on person.id = interview.person_id
where person.name = 'Jeremy Bowers'
"""
pd.read_sql_query(jeremy_testimony,con)

,transcript
0,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n"


The real culprit has not yet been caught.

In [67]:
suspect = ''' 
select person.name
from person 
join drivers_license on person.license_id = drivers_license.id
where 
drivers_license.car_model = 'Model S' and car_make = 'Tesla' and gender = 'female' and hair_color = 'red'
'''
pd.read_sql_query(suspect,con)

,name
0,Red Korb
1,Regina George
2,Miranda Priestly


i have 3 suspect so i need further insvestigation.

In [70]:
Culprit = ''' 
select name
from person
join facebook_event_checkin on person.id = facebook_event_checkin.person_id
where event_name = 'SQL Symphony Concert' and (name = 'Red Korb' or name = 'Regina George' or name = 'Miranda Priestly')

'''
pd.read_sql_query(Culprit,con)

,name
0,Miranda Priestly
1,Miranda Priestly
2,Miranda Priestly


Miranda Priestly is the real culprit